# Project Demo

### Implemented as a package

In [1]:
import piano_reduction.tools as pr
import piano_reduction.compute_features as cf
from piano_reduction.classes import ScoreData, Reduction

Using TensorFlow backend.


### Used Python Package **Music21**

In [12]:
score = pr.load_score('demo', 2)
score

demo/Symphony_No._9_2nd_Movement_segment.musicxml


<music21.stream.Score 0x13bed5f98>

## Preprocessor
### Convert Score to Dataframe

In [13]:
dfs = pr.load_df('demo', 2, use_cache=False)
dfs[0].head()

Creating df/demo/Symphony_No._9_2nd_Movement_segment.musicxml
P1: Flauti: Fløyte 0
P2: Oboi: Obo 0
P3: Clarinetti in C: Klarinett 0
P4: Fagotti: Fagott 0
P5: Corni in D: D-horn -10
P6: Corni in B: B♭-horn -2
P7: Trombe in D: D-trompet 2
P8: Tromboni Alto e Tenore: Alttrombone 0
P9: Tromboni Basso: Basstrombone 0
P10: Timpani in F: Pauker 0
P11: Violino I: Fioliner 0
P12: Violino II: Fioliner 0
P13: Viola: Bratsjer 0
P14: Violoncello: Celloer 0
P15: Basso: Kontrabasser 0


,measure,offset,part,pitch,ps,duration,tie,chosen,color
12,0.0,0.0,22,D3,50.0,<music21.duration.Duration 1.5>,None,1,None
9,0.0,0.0,21,D4,62.0,<music21.duration.Duration 1.5>,None,1,None
6,0.0,0.0,20,D5,74.0,<music21.duration.Duration 1.5>,None,1,None
0,0.0,0.0,17,D6,86.0,<music21.duration.Duration 1.5>,None,1,None
13,0.0,1.5,22,D2,38.0,<music21.duration.Duration 0.5>,None,1,None


### Compute Features for each note

In [14]:
data = ScoreData(dfs)
data.df.head()

,measure,offset,part,pitch,ps,duration,tie,chosen,color,y_train,...,in_chord,lowest,occurrence,offset_value,onset_after_rest,pitch_distance,rhythm_variety,strong_beats,sustained_rhythm,vertical_doubling
12,0.0,0.0,22,D3,50.0,<music21.duration.Duration 1.5>,None,1,None,0.0,...,0.0,1.0,1.0,0.0,0.0,10.0,1.0,1.0,1.0,1.0
9,0.0,0.0,21,D4,62.0,<music21.duration.Duration 1.5>,None,1,None,0.0,...,0.0,0.0,1.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0
6,0.0,0.0,20,D5,74.0,<music21.duration.Duration 1.5>,None,1,None,0.0,...,0.0,0.0,1.0,0.0,0.0,14.0,1.0,1.0,1.0,1.0
0,0.0,0.0,17,D6,86.0,<music21.duration.Duration 1.5>,None,1,None,0.0,...,0.0,0.0,1.0,0.0,0.0,26.0,1.0,1.0,1.0,1.0
13,0.0,1.5,22,D2,38.0,<music21.duration.Duration 0.5>,None,1,None,0.0,...,0.0,1.0,1.0,1.5,0.0,22.0,1.0,0.0,1.0,1.0


In [15]:
data.save('score_data/demo.pkl')

In [4]:
data = ScoreData.load('score_data/demo.pkl')

## Machine Learning Component

### Load trained **Keras** model

In [2]:
reduction = Reduction.load('models/20190122/GRU with features 4.pkl')
reduction.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 200)               1195800   
_________________________________________________________________
activation_1 (Activation)    (None, 200)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               25728     
_________________________________________________________________
activation_2 (Activation)    (None, 128)               0         
Total params: 1,221,528
Trainable params: 1,221,528
Non-trainable params: 0
_________________________________________________________________


### Generate reduction from the model

In [3]:
data = ScoreData.load('score_data/demo.pkl')
new_data = reduction.predict_2(data, threshold=0.1)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [7]:
new_data.df.head()

,measure,offset,part,pitch,ps,duration,tie,chosen,color,y_train,...,lowest,occurrence,offset_value,onset_after_rest,pitch_distance,rhythm_variety,strong_beats,sustained_rhythm,vertical_doubling,y_pred
12,0.0,0.0,4,C3,48.0,<music21.duration.Duration 2.0>,None,1,None,0.0,...,1.0,0.0,0.0,0.0,12.0,1.0,1.0,0.0,1.0,1
17,0.0,0.0,-2,G3,55.0,<music21.duration.Duration 2.0>,None,1,None,0.0,...,0.0,1.0,0.0,0.0,12.0,1.0,1.0,0.0,1.0,1
0,0.0,0.0,0,C4,60.0,<music21.duration.Duration 1.0>,None,1,None,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1
8,0.0,0.0,2,E-4,63.0,<music21.duration.Duration 2.0>,None,1,None,0.0,...,0.0,0.0,0.0,0.0,3.0,1.0,1.0,1.0,1.0,1
6,0.0,0.0,1,G4,67.0,<music21.duration.Duration 2.0>,None,1,None,0.0,...,0.0,0.0,0.0,0.0,7.0,1.0,1.0,0.0,0.0,1


## Postprocessor
### Convert Dataframe back to Score

In [20]:
new_data.df['Comparison'] = new_data.get_color(new='y_pred', old='x_train')
new_data.df['Original'] = new_data.df['x_train']
new_data.df['Prediction'] = new_data.df['y_pred']
new_data.show_score(['Original', 'Prediction'], ['Comparison'])

In [11]:
new_data.show_score(['Prediction'])

In [11]:
data = ScoreData.load('score_data/demo.pkl')
new_data = reduction.predict(data, threshold=0.1)

In [12]:
new_data.df['Comparison'] = new_data.get_color(new='y_pred', old='x_train')
new_data.df['Original'] = new_data.df['x_train']
new_data.df['Prediction'] = new_data.df['y_pred']
new_data.show_score(['Original', 'Prediction'], ['Comparison'])